In [90]:
import os
import numpy as np
import tensorly as tl
from tensorly.decomposition import parafac
import datetime
import csv

In [91]:
# Get all the behaviour file name in the folder and store into a list
path = "/Users/Ryan/Desktop/tensorly/data"
files = os.listdir(path)
files.remove('.DS_Store')

In [92]:
# Tensor decomposition 

def tensor_decomposition(t, rank, files, n):
    # Choose n users to do the tensor decomposition
    files = files[0:n]
    # Find the biggest segment ID in all users behaviour
    biggest_segment_id_arr = []
    users = []
    
    # Read all users appearence behaviour from the file in test folder
    for file in files:
        a = np.loadtxt(path + "/" + file,'int32')
        users.append(a)
        biggest_segment = a.max()
        biggest_segment_id_arr.append(biggest_segment)
    
    # Generate a new tensor of given shape with zeros.
    segment_count = max(biggest_segment_id_arr)
    user_count = len(users)
    time_interval = 48

    t = np.zeros((user_count, segment_count, time_interval))
    
    # Put the user behaviour value into the tensor
    for idx, user in enumerate(users):
        for behaviour in user:
            row = behaviour[0]-1
            col = behaviour[1]
            val = behaviour[2]
            t[idx][row][col] = val
    
    start = datetime.datetime.now()
    X = tl.tensor(t, dtype=tl.float32)
    factors, errors = parafac(t, rank=rank, return_errors=True, n_iter_max = 200)
    end = datetime.datetime.now()
    time = (end - start).total_seconds()
    iteration = len(errors)
    result = [user_count, segment_count, time_interval, rank, iteration, errors[-1], time]
    return result

In [95]:
# Experiment in different rank
results = [["user", "segment", "interval", "rank", "iteration", "error", "time"]]
user_number = [5, 10]
ranks = [2, 5]
for n in user_number:
    for rank in ranks:
        result = tensor_decomposition(t, rank, files, n)
        results.append(result)

In [96]:
# Write the experiment results into the output file
with open('output.csv', 'w') as resultFile:
    wr = csv.writer(resultFile, dialect='excel')
    wr.writerows(results)